In [14]:
# processes historical and current built-up data from World Settlement Footprint

In [1]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
from rasterio.features import shapes
from os.path import exists
from pathlib import Path
from shutil import copyfile
import requests

In [2]:
country = os.getcwd().split('\\')[-1]

In [3]:
aoi_folder = Path('data/AOI')
output_folder = Path('data')
cities = pd.read_csv('centroids.csv').city
centroids = pd.read_csv('centroids.csv')
epsg_dict = dict(zip(centroids.city, centroids.utm))
wsf_types = ['evolution', '2019']

In [4]:
# download the WSF tiles that overlap with the AOIs
def download_wsf(city, wsf_type):
    data_folder = Path(r'F:\World Bank\City Scan') / country / ('data/WSF' + wsf_type)  # change file path as needed
    try:
        os.mkdir(data_folder)
    except FileExistsError:
        pass
    
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    shp_name = city_no_space + '_AOI.shp'
    shp = gpd.read_file(city / aoi_folder / shp_name)
    shp_bounds = shp.bounds
    
    for i in range(len(shp_bounds)):
        for x in range(math.floor(shp_bounds.minx[i] - shp_bounds.minx[i] % 2), math.ceil(shp_bounds.maxx[i]), 2):
            for y in range(math.floor(shp_bounds.miny[i] - shp_bounds.miny[i] % 2), math.ceil(shp_bounds.maxy[i]), 2):
                file_name = 'WSF' + wsf_type + '_v1_' + str(x) + '_' + str(y)
                if not exists(data_folder / (file_name + '.tif')):
                    if wsf_type == 'evolution':
                        file = requests.get('https://download.geoservice.dlr.de/WSF_EVO/files/' + file_name + '/' + file_name + '.tif')
                    elif wsf_type == '2019':
                        file = requests.get('https://download.geoservice.dlr.de/WSF2019/files/' + file_name + '.tif')
                    open(data_folder / (file_name + '.tif'), 'wb').write(file.content)

In [5]:
for wsf_type in wsf_types:
    for city in cities:
        download_wsf(city, wsf_type)

In [8]:
# merge the WSF tiles into one raster file
for wsf_type in wsf_types:
    try:
        raster_to_mosaic = []
        mosaic_file = 'WSF' + wsf_type + '.tif'

        mosaic_list = list((Path(r'F:\World Bank\City Scan') / country / 'data' / ('WSF' + wsf_type)).iterdir())
        for p in mosaic_list:
            raster = rasterio.open(p)
            raster_to_mosaic.append(raster)

        mosaic, output = merge(raster_to_mosaic)
        output_meta = raster.meta.copy()
        output_meta.update(
            {"driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": output,
            }
        )

        with rasterio.open(Path(r'F:\World Bank\City Scan') / country / 'data' / ('WSF' + wsf_type) / mosaic_file, 'w', **output_meta) as m:
            m.write(mosaic)
    except MemoryError:
        print(wsf_type)
        print('MemoryError. Try GIS instead for merging.')

evolution
MemoryError. Try GIS instead for merging.
2019
MemoryError. Try GIS instead for merging.


In [9]:
def clipdata_wsf(city, wsf_type):
    data_folder = Path(r'F:\World Bank\City Scan') / country / ('data/WSF' + wsf_type)
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    shp_name = city_no_space + '_AOI.shp'
    shp = gpd.read_file(city / aoi_folder / shp_name)
    features = shp.geometry
    
    input_raster = data_folder / ("WSF" + wsf_type + ".tif")
    with rasterio.open(input_raster) as src:
        out_image, out_transform = rasterio.mask.mask(
            src, features, crop=True)
        out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        output_4326_raster_clipped = city_lower + "_WSF" + wsf_type + "_4326.tif"

        with rasterio.open(city / output_folder / output_4326_raster_clipped, "w", **out_meta) as dest:
            dest.write(out_image)

In [10]:
def utm_wsf(city, wsf_type):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    shp_name = city_no_space + '_AOI.shp'
    crs = epsg_dict.get(city)
    shp = gpd.read_file(city / aoi_folder / shp_name).to_crs(epsg = crs)
    features = shp.geometry
    
    with rasterio.open(city / output_folder / (city_lower + "_WSF" + wsf_type + "_4326.tif")) as src:
        dst_crs = 'EPSG:' + str(crs)

        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(city / output_folder / (city_lower + '_WSF' + wsf_type + '_utm.tif'), 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    
    if wsf_type == 'evolution':
        with rasterio.open(city / output_folder / (city_lower + '_WSF' + wsf_type + '_utm.tif')) as src:
            out_image = src.read()
            pixelSizeX, pixelSizeY = src.res

        year_dict = {}
        for year in range(1985, 2016):
            if year == 1985:
                year_dict[year] = np.count_nonzero(
                out_image == year) * pixelSizeX * pixelSizeY / 1000000
            else:
                year_dict[year] = np.count_nonzero(
                out_image == year) * pixelSizeX * pixelSizeY / 1000000 + year_dict[year-1]

        # save CSV
        with open(city / output_folder / (city_lower + "_built_up_stats.csv"), 'w') as f:
            f.write("year,cumulative sq km\n")
            for key in year_dict.keys():
                f.write("%s,%s\n" % (key, year_dict[key]))

In [11]:
def reclass_wsf(city, wsf_type = 'evolution'):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    
    with rasterio.open(city / output_folder / (city_lower + '_WSF' + wsf_type + '_4326.tif')) as src:
        out_image = src.read()
        out_meta = src.meta.copy()
    
    out_image[0][out_image[0] < 1985] = 0
    out_image[0][(out_image[0] <= 2015) & (out_image[0] >= 2006)] = 4
    out_image[0][(out_image[0] < 2006) & (out_image[0] >= 1996)] = 3
    out_image[0][(out_image[0] < 1996) & (out_image[0] >= 1986)] = 2
    out_image[0][out_image[0] == 1985] = 1
    
    out_file = city_lower + '_WSF' + wsf_type + '_reclass.tif'
    with rasterio.open(city / output_folder / out_file, "w", **out_meta) as dest:
        dest.write(out_image)

In [12]:
def polygonize_wsf(city, wsf_type = '2019'):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    
    mask = None
    
    with rasterio.open(city / output_folder / (city_lower + '_WSF' + wsf_type + '_4326.tif')) as src:
        image = src.read(1)
        results = ({'properties': {'raster_val': v}, 'geometry': s} for i, (s, v) in enumerate(shapes(image, mask=mask, transform=src.transform)))
        geoms = list(results)
        gpd_polygonized_raster = gpd.GeoDataFrame.from_features(geoms)
        gpd_polygonized_raster = gpd_polygonized_raster[gpd_polygonized_raster.raster_val != 0]
    
    gpd_polygonized_raster.to_file(city / output_folder / (city_lower + '_WSF' + wsf_type + '_4326.shp'), crs = 'EPSG:4326')

In [13]:
for wsf_type in wsf_types:
    for city in cities:
        clipdata_wsf(city, wsf_type)
        if wsf_type == 'evolution':
            utm_wsf(city, wsf_type)
            reclass_wsf(city)
        if wsf_type == '2019':
            polygonize_wsf(city)

C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.